In [39]:
import numpy as np

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from NeuralNetNLayer import NeuralNet

**Load in and transform the dataset**

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.random.seed(1)

In [4]:
df = pd.read_csv('mnist/train.csv')

In [5]:
# Keep only the first two numbers (0 and 1) to convert this to binary classification
df_two = df[df['label'] <= 1]
df_two.label.value_counts()

1    4684
0    4132
Name: label, dtype: int64

In [6]:
# Check out the data - pixel values range from 0 to 255
df_two.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Split into X and y training and validation sets 
X_train, X_test, y_train, y_test = train_test_split(df_two.drop('label', axis=1), df_two['label'], train_size = 0.67)

In [32]:
# Standardize Before transposing (skelearn assumes obs x features format)
scaler = StandardScaler()
scaler = scaler.fit(X_train)
X_s = scaler.transform(X_train)
# Transpose X and convert to an array
X_t = X_s.T
X_t.shape

(784, 5906)

In [29]:
# Transpose y and convert to an array
y_t = y_train.values.reshape(1, len(y_train))
y_t.shape

(1, 5906)

**Create and train the model**

In [ ]:
from NeuralNetNLayer import NeuralNet

In [51]:
# Create and train the model with random initialization
clf = NeuralNet(X_ts, y_t, [6, 3, 1])

In [52]:
# Train
clf.train(0.5, 300)

Iteration: 0 , Cost, Accuracy 0.6931479673977137 43.10870301388419
Iteration: 100 , Cost, Accuracy 0.12309311684503893 99.77988486285135
Iteration: 200 , Cost, Accuracy 0.019428737685357255 99.96613613274636


In [61]:
# Create and train the model with he initialization
clf = NeuralNet(X_ts, y_t, [6, 3, 1], initialization='he')


In [62]:
clf.train(0.5, 300)

Iteration: 0 , Cost, Accuracy 0.691254282692146 46.83372841178463
Iteration: 100 , Cost, Accuracy 0.002396474533978421 99.81374873010498
Iteration: 200 , Cost, Accuracy 0.0017580764607663206 99.8306806637318


**Test accuracy on the validation dataset**

In [45]:
# Standardize (using the original values) and transform
X_s = scaler.transform(X_test)
X_v = X_s.T
X_v.shape

(784, 2910)

In [46]:
y_v = y_test.values.reshape(1, len(y_test))
y_v.shape

(1, 2910)

In [49]:
clf.validation_accuracy(X_v=X_v, y_v=y_v)

98.65979381443299